In [46]:
import pandas as pd

In [47]:
OSM_data = pd.read_json('amenities-vancouver.json', lines=True)

In [48]:
chains = OSM_data[OSM_data['tags'].apply(lambda tags: 'brand:wikidata' in tags)]

In [53]:
fast_food = chains[chains['amenity']=='fast_food']

In [54]:
restaurants = chains[chains['amenity']=='restaurant']

In [43]:
# https://ramiro.org/notebook/us-presidents-causes-of-death/
import requests

query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?restaurant ?followers WHERE {
    VALUES ?storetype {wd:Q18534542 wd:Q18509232}
    ?pid wdt:P31 ?storetype .
    ?pid wdt:P8687 ?followers .

    OPTIONAL {
        ?pid rdfs:label ?restaurant filter (lang(?restaurant) = "en") .
    }
    OPTIONAL {
        ?cid rdfs:label ?followers filter (lang(?followers) = "en") .
    }
}'''

url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
data = requests.get(url, params={'query': query, 'format': 'json'}).json()

In [77]:
# it would be desirable to use 'from_dict' here, but dtype cannot be set to 'dict'
chain_foods = []
for item in data['results']['bindings']:
    chain_foods.append({
        'name': item['restaurant']['value'],
        'followers': item['followers']['value']})

wiki_df = pd.DataFrame(chain_foods)
wiki_df['followers'] = (wiki_df['followers']).astype(int)
wiki_df.head()

,name,followers
0,Domino's Pizza,330000
1,Domino's Pizza,330000
2,Wasabi,4800
3,Eat'n Park,6700
4,MOS Burger,10500


In [67]:
wiki_fast_food = fast_food.merge(wiki_df, on='name')
wiki_restaurants = restaurants.merge(wiki_df, on='name')

In [69]:
# sample test for all Vancouver restaurants & fast food
from scipy import stats
xA = wiki_fast_food['followers']
xB = wiki_restaurants['followers']

In [70]:
print(stats.normaltest(xA).pvalue)
print(stats.normaltest(xB).pvalue)
print(stats.levene(xA, xB).pvalue)

3.431272688793852e-23
4.969296669914804e-56
0.4047997244282754


In [73]:
import numpy as np
ya_transf = np.log(xA)
yb_transf = np.log(xB)
print(stats.normaltest(ya_transf).pvalue)
print(stats.normaltest(yb_transf).pvalue)

0.00026216547557884743
0.0


In [76]:
print(stats.mannwhitneyu(xA, xB).pvalue)

0.032463570401136406
